In [11]:
import pandas as pd

#load the dataset
dataset = pd.read_csv('/Users/jamie/Desktop/Projects/spotify_songs.csv')

To clean the dataset, I handled missing values and duplicates carefully. 
For missing data in track_name, track_album_name, and track_artist, I didn’t drop rows since each song can still be uniquely identified by its track_id.
To handle duplicates in the dataset, I chose to calculate the mean of variables and others for each unique track_id.

In [12]:
dataset.fillna({
    'track_name': 'Unknown',
    'track_artist': 'Unknown',
    'track_album_name': 'Unknown',
    'playlist_name': 'Unknown',
    'playlist_genre': 'Unknown',
    'playlist_subgenre': 'Unknown',
    'track_popularity': 0,
    'valence': dataset['valence'].mean(),
    'energy': dataset['energy'].mean(),
    'danceability': dataset['danceability'].mean(),
    'loudness': dataset['loudness'].mean(),
    'speechiness': dataset['speechiness'].mean(),
    'acousticness': dataset['acousticness'].mean(),
    'instrumentalness': dataset['instrumentalness'].mean(),
    'liveness': dataset['liveness'].mean(),
    'tempo': dataset['tempo'].mean(),
    'key': dataset['key'].mean(),
    'duration_ms': dataset['duration_ms'].mean()
}, inplace=True)

spotify_cleaned = dataset.groupby('track_id').agg({
    'track_name': 'first',  # Retain the first occurrence
    'track_artist': 'first',
    'track_album_name': 'first',
    'playlist_name': 'first',
    'playlist_genre': 'first',
    'playlist_subgenre': 'first',
    'track_popularity': 'mean',
    'valence': 'mean',
    'energy': 'mean',
    'danceability': 'mean',
    'loudness': 'mean',
    'speechiness': 'mean',
    'acousticness': 'mean',
    'instrumentalness': 'mean',
    'liveness': 'mean',
    'tempo': 'mean',
    'key': 'mean',
    'duration_ms': 'mean'
}).reset_index()

To ensure consistency and improve model performance, I scaled all the variables except the key feature I am using normalization. Scaling adjusts each variable to have a mean of 0 and a standard deviation of 1,
and one hot the key values instead because it's a categorical column.

In [15]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# One-hot encode the 'key' column
one_hot = pd.get_dummies(spotify_cleaned['key'], prefix='Key')
one_hot = one_hot.astype(int)

# Scale numerical columns
scaler = StandardScaler()
numerical_features = ['track_popularity', 'valence', 'energy', 'danceability', 
                      'loudness', 'speechiness', 'acousticness', 'instrumentalness', 
                      'liveness', 'tempo', 'duration_ms']
scaled_features = scaler.fit_transform(spotify_cleaned[numerical_features])

# Combine scaled features, one-hot encoding, and retained columns
scaled_df = pd.DataFrame(scaled_features, columns=numerical_features)
spotify_final = pd.concat([
    spotify_cleaned[['track_id', 'track_name', 'track_artist']],
    scaled_df,
    one_hot
], axis=1)



In [16]:
#check encoding
for column in spotify_final.select_dtypes(include=['object']).columns:
    spotify_final[column] = spotify_final[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
print(spotify_final.head())

                 track_id    track_name        track_artist  track_popularity  \
0  0017A6SJgTbfQVU2EtsPNo      Pangarap     Barbie's Cradle          0.070468   
1  002xjHwzEx66OWFV2IP9dk    The Others                RIKA         -1.026488   
2  004s3t0ONYlzxII9PLgU6z  I Feel Alive       Steady Rollin         -0.478010   
3  008MceT31RotUANsKuzy3L   Liquid Blue  The.madpix.project         -0.646772   
4  008rk8F6ZxspZT4bUlkIQG         Fever         YOSA & TAAR         -0.056104   

    valence    energy  danceability  loudness  speechiness  acousticness  ...  \
0  0.237324 -1.620642      0.196372 -1.070521    -0.822526      0.457022  ...   
1  0.800618  0.030586     -0.489580  0.189611    -0.714290     -0.503036  ...   
2 -0.453992  0.989715     -2.403387  0.684640    -0.621657     -0.742714  ...   
3  1.457794  0.521049      0.038603  0.386569    -0.526097     -0.791812  ...   
4 -0.061393  0.760832      0.059182  0.170508    -0.566076     -0.283556  ...   

   Key_2.0  Key_3.0  Key_4

In [17]:
#save the cleaned dataset
output_file = 'final_spotify_songs.csv'
spotify_final.to_csv(output_file, index=False, encoding='utf-8')

Let's calculate the correlation between track_popularity and other variables.

In [26]:
import pymysql
from sqlalchemy import create_engine

#mySQL connection
connection_string = "mysql+pymysql://root:<password>@localhost/spotify_db"
engine = create_engine(connection_string)

#query the data
query = "SELECT track_popularity, valence, energy, danceability, loudness, speechiness, acousticness, instrumentalness, liveness, tempo, duration_ms, Key_0, Key_1, Key_2, Key_3, Key_4, Key_5, Key_6, Key_7, Key_8, Key_9, Key_10, Key_11 FROM spotify_data"
df = pd.read_sql(query, engine)

#correlation
correlations = df.corr()

#correlations with track_popularity
print(correlations['track_popularity'].sort_values(ascending=False))

track_popularity    1.000000
acousticness        0.091725
danceability        0.046597
loudness            0.037285
valence             0.022581
Key_8               0.021443
Key_0               0.015165
speechiness         0.005205
Key_1               0.005170
tempo               0.004446
Key_5               0.003276
Key_10              0.000330
Key_9              -0.002198
Key_3              -0.003248
Key_4              -0.003536
Key_6              -0.004717
Key_11             -0.006497
Key_2              -0.008473
Key_7              -0.017476
liveness           -0.052773
energy             -0.103622
instrumentalness   -0.124431
duration_ms        -0.131202
Name: track_popularity, dtype: float64


In [27]:
#multiple linear regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

#features and target variable
X = df.drop(columns=['track_popularity'])
y = df['track_popularity']

#split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

#predict on the test set
y_pred = model.predict(X_test)

#evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

#display feature coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

print(coefficients)


Mean Squared Error: 0.9493873403818097
R^2 Score: 0.04326716300172373
             Feature  Coefficient
3           loudness     0.158676
18             Key_8     0.053381
5       acousticness     0.044203
10             Key_0     0.043577
8              tempo     0.033722
20            Key_10     0.022714
0            valence     0.019387
2       danceability     0.019049
11             Key_1     0.011575
14             Key_4     0.004088
15             Key_5     0.001906
21            Key_11    -0.007882
12             Key_2    -0.009053
19             Key_9    -0.015862
16             Key_6    -0.016083
4        speechiness    -0.030938
7           liveness    -0.031240
17             Key_7    -0.036775
13             Key_3    -0.051588
6   instrumentalness    -0.089818
9        duration_ms    -0.109736
1             energy    -0.189833


From the result, the high mean squared error and low R squared score indicate that the linear regression model is not performing well in predicting track popularity. It could be other variables, not in the dataset, like artist popularity, marketing other factors could be predictors. Also, popularity might have nonlinear dependencies so it doesn't perfom well by a simple linear model. I can try nonlinear models such as random forest or polynomial regression.